In [0]:
import requests
import os
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import to_timestamp, col, explode
import delta

token = dbutils.secrets.get(scope="meu-escopo", key="token_brapi")
tickers = ["JHSF3"]
#"UNIP6", "PETR4", "VALE3", "ITUB4", "BBAS3", "BBDC4", "ITSA4", "RANI3", "EGIE3", "PRIO3","BRAV3"
schema = StructType([StructField("Simbolo", StringType(), False),
                     StructField("Nome", StringType(), True),
                     StructField("Moeda", StringType(), True),
                     StructField("Fechamento_Anterior", DoubleType(), True),
                     StructField("Cotacao", DoubleType(), True),
                     StructField("Intervalo_52_Semanas", StringType(), True),
                     StructField("Data", StringType(), True)
                     ])

for ticker in tickers:
    #api brabi
    url = f"https://brapi.dev/api/quote/{ticker}?token={token}"
   
    response = requests.get(url,params={"range": "3mo", "interval": "1d"})

    if response.status_code == 200:
        dados_brapi = response.json()["results"][0]
   
        # Cria o DataFrame Spark
        data = [(dados_brapi["symbol"],
                dados_brapi["shortName"],
                dados_brapi["currency"],
                dados_brapi["regularMarketPreviousClose"],
                dados_brapi["regularMarketPrice"],
                dados_brapi["fiftyTwoWeekRange"],
                dados_brapi["regularMarketTime"],
                dados_brapi["historicalDataPrice"])]

        df_temp = spark.createDataFrame(data, schema)
        df_spark = df_temp.withColumn("Data", to_timestamp(col("Data")))
        df_spark_hist = df_spark.select(col("symbol")).select(explode(col("historicalDataPrice")))
        
        if spark.catalog.tableExists("transacional.cotacoes_db.cotacoes"):

            transacional_cotacoes = delta.DeltaTable.forName(spark, "transacional.cotacoes_db.cotacoes")
            
            (transacional_cotacoes.alias("r")
                        .merge(df_spark.alias("s"),"r.Simbolo = s.Simbolo")
                        .whenMatchedUpdateAll() # Se encontrar o Simbolo, atualiza todas as colunas
                        .whenNotMatchedInsertAll() # Se não encontrar, insere a nova linha
                        .execute()
                        )
        else:
            (df_spark.writeTo("transacional.cotacoes_db.cotacoes")
                     .tableProperty("delta.autoOptimize.optimizeWrite", "true")
                     .tableProperty("delta.autoOptimize.autoCompact", "true")
                     .tableProperty("delta.enableChangeDataFeed", "true")
                     .using("delta")
                     .create()
                     )
        
        (df_spark_hist.write
                     .format("delta")
                     .tableProperty("delta.autoOptimize.optimizeWrite", "true")
                     .tableProperty("delta.autoOptimize.autoCompact", "true")
                     .tableProperty("delta.enableChangeDataFeed", "true")
                     .mode("append")
                     .saveAsTable("transacional.cotacoes_db.hist_cotacoes")
                     )
        
        #print(f"Coleta realizada com sucesso, status code: {response.status_code}")
        #display(df_spark)
    
    else:
        print(f"Erro: {response.status_code}")

In [0]:
import requests
import os
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, LongType
from pyspark.sql.functions import to_timestamp, col, explode, from_unixtime
import delta

token = dbutils.secrets.get(scope="meu-escopo", key="token_brapi")
tickers = ["VALE3"]
#"UNIP6", "PETR4", "VALE3", "ITUB4", "BBAS3", "BBDC4", "ITSA4", "RANI3", "EGIE3", "PRIO3","BRAV3"
schema_historico = StructType([StructField("date", StringType(), True),
                               StructField("open", DoubleType(), True),
                               StructField("high", DoubleType(), True),
                               StructField("low", DoubleType(), True),
                               StructField("close", DoubleType(), True),
                               StructField("volume",LongType(), True)                            
                               ])
schema = StructType([StructField("Simbolo", StringType(), False),
                     StructField("Nome", StringType(), True),
                     StructField("Moeda", StringType(), True),                     
                     StructField("Cotacao", DoubleType(), True),
                     StructField("Intervalo_52_Semanas", StringType(), True),
                     StructField("Data", StringType(), True),
                     StructField("Historico_cotacao", ArrayType(schema_historico), True)
                     ])

for ticker in tickers:
    #api brabi
    url = f"https://brapi.dev/api/quote/{ticker}?token={token}"
   
    response = requests.get(url,params={"range": "3mo", "interval": "1d"})

    if response.status_code == 200:
        dados_brapi = response.json()["results"][0]
   
        # Cria o DataFrame Spark
        data = [(dados_brapi["symbol"],
                dados_brapi["shortName"],
                dados_brapi["currency"],                
                dados_brapi["regularMarketPrice"],
                dados_brapi["fiftyTwoWeekRange"],
                dados_brapi["regularMarketTime"],
                dados_brapi["historicalDataPrice"])]

        df_temp = spark.createDataFrame(data, schema)
        df_spark = df_temp.withColumn("Data", to_timestamp(col("Data"))).drop("Historico_cotacao")
        df_spark_hist = df_temp.select(col("Simbolo"),(explode(col("Historico_cotacao")).alias("Historico"))).select("Simbolo","Historico.*")
        df_spark_hist = df_spark_hist.withColumn("date", from_unixtime(col("date")).cast("timestamp"))
display(df_spark_hist)
display(df_spark)
display(dados_brapi)


    
    

In [0]:
import requests
import os
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, LongType
from pyspark.sql.functions import to_timestamp, col, explode
import delta

token = dbutils.secrets.get(scope="meu-escopo", key="token_brapi")
tickers = ["JHSF3"]
#"UNIP6", "PETR4", "VALE3", "ITUB4", "BBAS3", "BBDC4", "ITSA4", "RANI3", "EGIE3", "PRIO3","BRAV3"
schema_historico = StructType([StructField("date", StringType(), True),
                               StructField("open", DoubleType(), True),
                               StructField("high", DoubleType(), True),
                               StructField("low", DoubleType(), True),
                               StructField("close", DoubleType(), True),
                               StructField("volume", LongType(), True),
                               StructField("adjustedClose", DoubleType(), True)
                               ])
schema = StructType([StructField("Simbolo", StringType(), False),
                     StructField("Nome", StringType(), True),
                     StructField("Moeda", StringType(), True),
                     StructField("Fechamento_Anterior", DoubleType(), True),
                     StructField("Cotacao", DoubleType(), True),
                     StructField("Intervalo_52_Semanas", StringType(), True),
                     StructField("Data", StringType(), True),
                     StructField("Historico_cotacao", ArrayType(schema_historico), True)
                     ])

for ticker in tickers:
    #api brabi
    url = f"https://brapi.dev/api/quote/{ticker}?token={token}"
   
    response = requests.get(url,params={"range": "3mo", "interval": "1d"})

    if response.status_code == 200:
        dados_brapi = response.json()["results"][0]
display(dados_brapi)